# Comparisons for ALFA_PLUS (2017-05-10)

In [31]:
# After running find . > $file on both archives (BSC and UPCnet)
upcnet = open('/tmp/xnat_stats/ALFA_PLUS_upcnet_20170510.txt').readlines()
bsc = open('/tmp/xnat_stats/ALFA_PLUS_bsc_20170510.txt').readlines()

In [32]:
from glob import glob
import pandas as pd
import os.path as osp
upc_dcm = [(e.split('/')[-3], osp.basename(e)[:-1]) for e in upcnet if e.endswith('.dcm\n')]
bsc_dcm = [(e.split('/')[-3], osp.basename(e)[:-1]) for e in bsc if e.endswith('.dcm\n')]

### Display the first identified DICOM files

In [33]:
print upc_dcm[:10], bsc_dcm[:10]

[('1001', '1.3.46.670589.11.78036.5.0.2668.2017011014071866000-1001-325-jmuk6y.dcm'), ('1001', '1.3.46.670589.11.78036.5.0.2668.2017011014071866000-1001-277-ko5ex4.dcm'), ('1001', '1.3.46.670589.11.78036.5.0.2668.2017011014071866000-1001-104-k4uoiq.dcm'), ('1001', '1.3.46.670589.11.78036.5.0.2668.2017011014071866000-1001-53-iqcs13.dcm'), ('1001', '1.3.46.670589.11.78036.5.0.2668.2017011014071866000-1001-232-jm6xo9.dcm'), ('1001', '1.3.46.670589.11.78036.5.0.2668.2017011014071866000-1001-183-i87495.dcm'), ('1001', '1.3.46.670589.11.78036.5.0.2668.2017011014071866000-1001-291-k88n38.dcm'), ('1001', '1.3.46.670589.11.78036.5.0.2668.2017011014071866000-1001-149-ircjat.dcm'), ('1001', '1.3.46.670589.11.78036.5.0.2668.2017011014071866000-1001-216-hdzpg9.dcm'), ('1001', '1.3.46.670589.11.78036.5.0.2668.2017011014071866000-1001-348-irxlw2.dcm')] [('1001', '1.3.46.670589.11.78036.5.0.2152.2016121314040306010-1001-233-rgk1h9.dcm'), ('1001', '1.3.46.670589.11.78036.5.0.2152.2016121314040306010-10

### Numbers of existing DICOM files on both sides:

In [34]:
print 'upc:', len(upc_dcm), 'bsc:', len(bsc_dcm)

upc: 1078174 bsc: 1056847


Numbers of non-dicom items:

In [35]:
nondcm_upcnet = [e for e in upcnet if not '.dcm' in e]
nondcm_bsc = [e for e in bsc if not '.dcm' in e]
print 'upc:', len(nondcm_upcnet), 'bsc:', len(nondcm_bsc)

upc: 4428 bsc: 7471


Dictionaries by sequences:

In [36]:
# Creating dictionaries
upc_dict = {}
bsc_dict = {}

for seq, dcmfile in upc_dcm:
    upc_dict.setdefault(seq, []).append(dcmfile)
for seq, dcmfile in bsc_dcm:
    bsc_dict.setdefault(seq, []).append(dcmfile)

## Example of difference for a given sequence:

In [37]:
s = '601'
mismatch_ex = list(set(upc_dict[s]).difference(set(bsc_dict[s])))[0]
test = [e for e in upcnet if mismatch_ex in e]

print 'Example of mismatching file:', mismatch_ex

print 'is %s found in upcnet : %s, and in BSC: %s'%(test[0], test[0] in upcnet, test[0] in bsc)

Example of mismatching file: 1.3.46.670589.11.78036.5.0.2668.2017011016501692004-601-6153-w610xw.dcm
is ./11273/SCANS/601/DICOM/1.3.46.670589.11.78036.5.0.2668.2017011016501692004-601-6153-w610xw.dcm
 found in upcnet : True, and in BSC: False


Another one

In [38]:
s = '701'
mismatch_ex = list(set(upc_dict[s]).difference(set(bsc_dict[s])))[0]
test = [e for e in upcnet if mismatch_ex in e]

print 'Example of mismatching file:', mismatch_ex

print 'is %s found in upcnet : %s, and in BSC: %s'%(test[0], test[0] in upcnet, test[0] in bsc)

Example of mismatching file: 1.3.46.670589.11.78036.5.0.2668.2017011016501692004-701-96-1hu8r0t.dcm
is ./11273/SCANS/701/DICOM/1.3.46.670589.11.78036.5.0.2668.2017011016501692004-701-96-1hu8r0t.dcm
 found in upcnet : True, and in BSC: False


## Matching files between servers for every sequence:

The good thing is that it appears that all files from BSC find their match on UPCnet.

Yet the opposite is not true.

In [39]:
results = []
print set(upc_dict.keys()).difference(set(bsc_dict.keys())), 'are in UPC, missing in BSC'
print set(bsc_dict.keys()).difference(set(upc_dict.keys())), 'are in BSC, missing in UPC'
common_sq = set(bsc_dict.keys()).intersection(set(upc_dict.keys()))
print len(common_sq), 'in common'

for each in common_sq:
    row = []
    row.append(len(upc_dict[each]))
    row.append(len(bsc_dict[each]))
    row.append(len(set(upc_dict[each]).difference(set(bsc_dict[each]))))
    row.append(len(set(bsc_dict[each]).difference(set(upc_dict[each]))))
    row.append(len(upc_dict[each]) - len(bsc_dict[each]))
    results.append(row)
pd.DataFrame(results, index=common_sq, columns=['UPCnet', 'BSC', 'upc.diff(bsc)', 'bsc.diff(upc)', 'UPCnet - BSC (raw diff in numbers)'])

set([]) are in UPC, missing in BSC
set([]) are in BSC, missing in UPC
51 in common


,UPCnet,BSC,upc.diff(bsc),bsc.diff(upc),UPCnet - BSC (raw diff in numbers)
1304,90,90,0,0,0
601,676349,662548,13801,0,13801
1301,223315,223248,67,0,67
603,55,55,0,0,0
602,3,3,0,0,0
1201,3643,3564,79,0,79
701,11075,10849,226,0,226
702,1,1,0,0,0
1202,1,1,0,0,0
1901,4753,4753,0,0,0
